<a href="https://colab.research.google.com/github/AminHosseini96/CS-SBU-NeuralNetwork_MSc2021-projects/blob/main/AminHosseini-99422057/project03/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import shutil
import cv2
import random
from google.colab.patches import cv2_imshow
from PIL import Image ,ImageOps
import os, sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn.functional as F
from skimage import io
from skimage.color import rgb2lab,rgb2gray,lab2rgb
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# Colorizing Grayscale Images

## Resizing Images

In [ ]:
def resize(path):
  counter = 1
  dirs = os.listdir(path)
  os.makedirs(path + 'resized/', exist_ok=True)
  for item in dirs:
    if os.path.isfile(path+item):
      print(counter, item)
      im = Image.open(path+item)
      filename, extension = os.path.splitext(item)
      if extension == '.png':
        im = im.convert('RGB')
      imResize = im.resize((200,200), Image.ANTIALIAS)
      imResize.save(path + 'resized/' + filename + ' (resized).jpg', 'JPEG', quality=90)
    counter +=1

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/training_set/drawings/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/training_set/iconography/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/training_set/painting/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/training_set/sculpture/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/training_set/engraving/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/validation_set/drawings/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/validation_set/engraving/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/validation_set/iconography/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/validation_set/painting/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/validation_set/sculpture/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/training_set/drawings/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/training_set/engraving/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/training_set/iconography/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/training_set/painting/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/training_set/sculpture/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/validation_set/drawings/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/validation_set/engraving/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/validation_set/iconography/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/validation_set/painting/'
resize(path)

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/validation_set/sculpture/'
resize(path)

In [ ]:
os.remove('/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/validation_set/sculpture/52.jpg')

In [ ]:
path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/training_set/'
dest = '/content/drive/My Drive/SBU/NN/03/data1/train/'
for dir in os.listdir(path):
  for file in os.listdir(path+dir+'/resized/'):
    print(file)
    shutil.copy(path+dir+'/resized/'+file, dest)

path = '/content/drive/My Drive/SBU/NN/03/data1/dataset/dataset_updated/validation_set/'
dest = '/content/drive/My Drive/SBU/NN/03/data1/test/'
for dir in os.listdir(path):
  for file in os.listdir(path+dir+'/resized/'):
    print(file)
    shutil.copy(path+dir+'/resized/'+file, dest)
    
path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/training_set/'
dest = '/content/drive/My Drive/SBU/NN/03/data1/train/'
for dir in os.listdir(path):
  for file in os.listdir(path+dir+'/resized/'):
    print(file)
    shutil.copy(path+dir+'/resized/'+file, dest)

path = '/content/drive/My Drive/SBU/NN/03/data1/musemart/dataset_updated/validation_set/'
dest = '/content/drive/My Drive/SBU/NN/03/data1/test/'
for dir in os.listdir(path):
  for file in os.listdir(path+dir+'/resized/'):
    print(file)
    shutil.copy(path+dir+'/resized/'+file, dest)

## Preparing Dataset

In [7]:
class ScanFile(object): 
    def __init__(self,directory,prefix=None,postfix=None):
        self.directory=directory
        self.prefix=prefix
        self.postfix=postfix
        
    def scan_files(self):  
        files_list=[]  
          
        for dirpath,dirnames,filenames in os.walk(self.directory): 
            '''
            dirpath is a string, the path to the directory.  
            dirnames is a list of the names of the subdirectories in dirpath (excluding '.' and '..').
            filenames is a list of the names of the non-directory files in dirpath.
            '''
            for special_file in filenames:  
                if self.postfix:  
                    special_file.endswith(self.postfix)  
                    files_list.append(os.path.join(dirpath,special_file))  
                elif self.prefix:  
                    special_file.startswith(self.prefix)
                    files_list.append(os.path.join(dirpath,special_file))  
                else:  
                    files_list.append(os.path.join(dirpath,special_file))  
                                
        return files_list  
    
    def scan_subdir(self):
        subdir_list=[]
        for dirpath,dirnames,files in os.walk(self.directory):
            subdir_list.append(dirpath)
        return subdir_list

In [8]:
class scenedataset(Dataset):
    def __init__(self,dir_list,flag):
        
        num=len(dir_list)
        
        if flag == "train":
            self.sub_dir_list = dir_list[:1000]
            print("train number:",len(self.sub_dir_list))
            self.trans=transforms.Compose([transforms.RandomHorizontalFlip(p=0.5)])
        if flag == "validation":
            self.sub_dir_list = dir_list
            print("validation number:",len(self.sub_dir_list))
            self.trans=transforms.Compose([transforms.RandomHorizontalFlip(p=0)])
        
    def __getitem__(self,idx):

        image_path = self.sub_dir_list[idx]
        image_raw = Image.open(image_path)
        image_raw = image_raw.convert('RGB')
        image_raw = image_raw.resize((32,32), Image.ANTIALIAS)
        image_original = self.trans(image_raw)
        image_original = np.asarray(image_original)
        # image_lab = (rgb2lab(image_original)+128)/255
        # image_ab = image_lab[:,:,1:3]
        # image_ab = torch.from_numpy(image_ab.transpose((2,0,1))).float()
        # image_original = rgb2gray(image_original)
        image_gray = rgb2gray(image_original)
        #image_original = torch.from_numpy(image_original.transpose((2,0,1))).float()
        image_original = torch.from_numpy(image_original).float()
        image_original = image_original.permute(2, 1, 0)
        image_gray = torch.from_numpy(image_gray).unsqueeze(0)        
        # data = image_original
        # label = image_ab
        label =image_original
        data = image_gray
        #print(label.max())
        
        return data,label
    
    def __len__(self):
        return len(self.sub_dir_list)

In [9]:
BATCH_SIZE = 5

data_dir = '/content/drive/My Drive/SBU/NN/03/data1/train/tr/'
dir_list = ScanFile(data_dir).scan_files()
dir_list = sorted(dir_list,key=lambda k:random.random())
# print(len(dir_list))

training_set = scenedataset(dir_list,flag="train")    
training_loader = torch.utils.data.DataLoader(training_set,batch_size=BATCH_SIZE,num_workers=0)

data_dir = '/content/drive/My Drive/SBU/NN/03/data1/test/ts/'
dir_list = ScanFile(data_dir).scan_files()
dir_list = sorted(dir_list,key=lambda k:random.random())
# print(len(dir_list))

validation_set=scenedataset(dir_list,flag="validation")
validation_loader = torch.utils.data.DataLoader(validation_set,batch_size=BATCH_SIZE,num_workers=0)

train number: 1000
validation number: 986


In [ ]:
for data, lable in validation_loader:
    print(data.shape, lable.shape)

torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) torch.Size([5, 3, 32, 32])
torch.Size([5, 1, 32, 32]) t

## Model Designe & Training

In [10]:
def reconstruct_rgb(grey_scale,ab_img):
         
         color_image = torch.cat((grey_scale, ab_img), 0).numpy() # combine channels
         color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
         color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
         color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
         color_image = lab2rgb(color_image.astype(np.float64))
         print(grey_scale.max())
         print(grey_scale.min())
         grey_scale = grey_scale.squeeze().numpy()
         plt.figure(1)
         plt.imshow(grey_scale,cmap='gray')
         plt.figure(2)
         plt.imshow(color_image)

In [11]:
class ConvNet(nn.Module):

    def __init__(self, batch_size):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.linear1 = nn.Linear(32*7*7, 512)
        self.linear2 = nn.Linear(512, 3072)

    def forward(self, x):
        # Encoder Phase
        p1 = F.max_pool2d(self.conv1(x), (2,2))
        p2 = F.max_pool2d(self.conv2(p1), (2,2))
        # Decoder Phase
        f1 = F.relu(self.linear1(p2.view(5,-1)))
        f2 = torch.sigmoid(self.linear2(f1)).view(-1, 3, 32, 32)
        return f2

In [12]:
model = ConvNet(BATCH_SIZE)

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


ConvNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (linear1): Linear(in_features=1568, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=3072, bias=True)
)

In [14]:
learning_rate = 0.01

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_loss_result={}
epoch_valid_loss_result={}
epochs =20

for epoch in range(epochs):

    epoch_loss=0
    epoch_valid_loss=0
    running_loss=0.0
    print("This is epoch",epoch+1)
    batch_num=0

    for data in training_loader:
        inputs, labels = data
        labels = labels.to(device,dtype=torch.float)
        optimizer.zero_grad()
        
        image=inputs.to(device,dtype=torch.float)
        outputs=model(image).to(device,dtype=torch.float)
        #print(outputs.shape)
       # print(labels.shape)
        loss=criterion(outputs,labels)
        
        loss.backward()
        optimizer.step()
        epoch_loss+=loss.item()
      #  step_loss+=loss.item()
        batch_num=batch_num+1
        
        running_loss = 0.0
        train_loss_result[epoch]=epoch_loss    
        with torch.no_grad():
            try:
                for i,data in enumerate(validation_loader):
                    inputs,label=data
                    label = label.to(device,dtype=torch.float)
                    image=inputs.to(device,dtype=torch.float)
                    outputs=model(image).to(device,dtype=torch.float)
                    valid_loss=criterion(outputs,label)
                    epoch_valid_loss += valid_loss.item()  
            except:
                continue
    epoch_valid_loss_result[epoch]=epoch_valid_loss  

train_epoch_value=list(train_loss_result.keys())
validation_epoch_value=list(epoch_valid_loss_result.keys())

train_loss_value=list(train_loss_result.values())
valid_loss_value=list(epoch_valid_loss_result.values())


In [ ]:
train_loss_container, test_loss_container = [], []
epochs = 15

for e in range(epochs):  # loop over the dataset multiple times

    train_loss = 0.0
    test_loss = 0.0
    
    for batch, train_data in enumerate(x_train):
        
        # get the inputs
        ip, op = train_data, y_train[batch]
        # zero the parameter gradients
        optimizer.zero_grad()

        model_op = net(ip)
        loss = criterion(model_op, op)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        for batch_test, test_data in enumerate(x_test, 0):
            # get the inputs
            ip_test, op_test = test_data, y_test[batch_test]
            # forward + backward + optimize
            model_op = net(ip_test)
            loss_test = criterion(model_op, op_test)
            test_loss += loss_test.item()
    
    train_loss_container.append(train_loss)
    test_loss_container.append(test_loss)
    print('\rEPOCH: {} | Train_loss: {} | Test_loss: {}'.format(e, train_loss, test_loss), end='')
print('\nFinished Training')

In [ ]:
plt.figure(1)
plt.plot(train_epoch_value,train_loss_value,)
plt.show()
plt.figure(2)
plt.plot(validation_epoch_value,valid_loss_value)
plt.show()

In [16]:
pic = Image.open('/content/drive/My Drive/SBU/NN/03/data1/train/tr/0006 (resized).jpg')

In [17]:
picg = ImageOps.grayscale(pic)

# Generating Fruit

In [ ]:
os.chdir('/content/drive/My Drive/SBU/NN/03/data2/')

In [ ]:
!unzip 03.zip

Archive:  2fol-fruits.zip
replace fruits-360/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: 